In [ ]:
from undetected_chromedriver import Chrome as uc_Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# UC Driver başlatma
driver = uc_Chrome()
print("✅ UC Driver başlatıldı.")

# İstenen sayfaya git
driver.get("https://help.3ds.com/")
print("✅ Siteye gidildi.")

# Accept All butonuna tıkla
try:
    accept_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Accept All")]'))
    )
    accept_all_button.click()
    print("✅ 'Accept All' butonuna tıklandı.")
except Exception as e:
    print(f"⚠ 'Accept All' bulunamadı veya tıklanamadı: {e}")

time.sleep(2)

# Username alanına "kullaniciadi" yaz
try:
    username_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "username"))
    )
    username_input.send_keys("irembircan")
    print("✅ Kullanıcı adı girildi.")
except Exception as e:
    print(f"⚠ Kullanıcı adı alanı bulunamadı: {e}")

time.sleep(1)

# Continue butonuna tıkla
try:
    continue_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="main-content"]/div[2]/div[1]/div[2]/form/div/input'))
    )
    continue_button.click()
    print("✅ Continue butonuna tıklandı.")
except Exception as e:
    print(f"⚠ Continue butonuna tıklanamadı: {e}")

time.sleep(2)

# Password alanına "sifre" yaz
try:
    password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "password"))
    )
    password_input.send_keys("Fnss2025!")
    print("✅ Şifre girildi.")
except Exception as e:
    print(f"⚠ Şifre alanı bulunamadı: {e}")

time.sleep(1)

# Log in butonuna tıkla
try:
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//input[@value="Log in"]'))
    )
    login_button.click()
    print("✅ Log in butonuna tıklandı.")
except Exception as e:
    print(f"⚠ Log in butonuna tıklanamadı: {e}")

# Biraz bekle
time.sleep(30)


from undetected_chromedriver import Chrome as uc_Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

df = pd.DataFrame(columns=["konum", "icerik"])  

def get_breadcrumb_and_content(driver):
    breadcrumb = []
    try:
        li_elements = driver.find_elements(By.XPATH, '//ul[@class="breadcrumb-drop"]//li/a')
        for li in li_elements:
            text = li.text.strip()
            if text:
                breadcrumb.append(text)
    except Exception as e:
        print(f"⚠ breadcrumb ul okunamadı: {e}")

    try:
        nav_items = driver.find_elements(By.XPATH, '//div[@class="breadcrumb-item nav-item"]')
        for nav in nav_items:
            text = nav.text.strip()
            if text:
                breadcrumb.append(text)
    except Exception as e:
        print(f"⚠ breadcrumb nav okunamadı: {e}")

    breadcrumb_str = " > ".join(breadcrumb)
    print("➡️ Breadcrumb yolu:", breadcrumb_str)

    # 🔥 content'i al
    content_text = ""
    try:
        content_div = driver.find_element(By.XPATH, '//div[@class="content"]')
        content_text = content_div.text.strip()
    except Exception as e:
        print(f"⚠ content div okunamadı: {e}")

    # ➕ DF'e ekle
    df.loc[len(df)] = [breadcrumb_str, content_text]


def crawl_ul(driver, ul_xpath, level=2):
    try:
        sub_items = driver.find_elements(By.XPATH, f'{ul_xpath}//li')
        print(f"🔍 level={level} içinde {len(sub_items)} li bulundu.")

        idx = 0
        while idx < len(sub_items):
            try:
                li = sub_items[idx]
                link = li.find_element(By.XPATH, './/a[@class="toc-item"][1]')
                
                driver.execute_script("arguments[0].scrollIntoView(true);", link)
                time.sleep(0.5)
                link.click()
                print(f"✅ level={level}, idx={idx+1}: '{link.text.strip()}' tıklandı.")

                time.sleep(1)
                get_breadcrumb_and_content(driver)
                time.sleep(0.5)

                next_level = level + 1
                sub_uls = li.find_elements(By.XPATH, f'.//ul[@level="{next_level}"]')
                for sub_ul in sub_uls:
                    sub_ul_xpath = get_xpath(sub_ul)
                    crawl_ul(driver, sub_ul_xpath, next_level)

                driver.back()
                time.sleep(0.5)

                idx += 1

            except Exception as e:
                print(f"⚠ level={level}, idx={idx+1} tıklanamadı: {e}")
                idx += 1

    except Exception as e:
        print(f"🚨 level={level} ul işlenirken hata: {e}")

def get_xpath(el):
    path = ''
    while el.tag_name != 'html':
        siblings = el.find_elements(By.XPATH, f'./preceding-sibling::{el.tag_name}')
        idx = len(siblings) + 1
        path = f'/{el.tag_name}[{idx}]' + path
        el = el.find_element(By.XPATH, '..')
    return '/html' + path

data_loc = 8
while data_loc <= 8:
    try:
        menu_item = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f'//li[@data-location="{str(data_loc)}"]'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", menu_item)
        time.sleep(0.5)
        menu_item.click()
        print(f"✅ data-location={data_loc} tıklandı.")
        time.sleep(1)

        crawl_ul(driver, f'//li[@data-location="{data_loc}"]//ul[@level="2"]', level=2)

    except Exception as e:
        print(f"⚠ data-location={data_loc} bulunamadı veya tıklanamadı: {e}")

    data_loc += 1

driver.quit()
print("✅ İşlem tamam, driver kapatıldı.")

# ➕ CSV'ye kaydet
df.to_csv("veri.csv", index=False)
print("✅ veri.csv kaydedildi.")

✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' butonuna tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue butonuna tıklandı.
✅ Şifre girildi.
✅ Log in butonuna tıklandı.
✅ data-location=2 tıklandı.
🔍 level=2 içinde 11 li bulundu.
✅ level=2, idx=1: 'Setting Up Your 3DEXPERIENCE Platform' tıklandı.
➡️ Breadcrumb yolu: Getting Started > Setting Up Your 3DEXPERIENCE Platform
✅ level=2, idx=2: 'Starting the 3DEXPERIENCE Platform' tıklandı.
➡️ Breadcrumb yolu: Getting Started > Starting the 3DEXPERIENCE Platform
✅ level=2, idx=3: 'Discovering Roles, Apps, Business Experiences, and Solutions Portfolio' tıklandı.
➡️ Breadcrumb yolu: Getting Started > Discovering Roles, Apps, Business Experiences, and Solutions Portfolio
✅ level=2, idx=4: 'Accessing a Dashboard' tıklandı.
➡️ Breadcrumb yolu: Getting Started > Accessing a Dashboard
✅ level=2, idx=5: 'Opening an App from the Compass' tıklandı.
➡️ Breadcrumb yolu: Getting Started > Opening an App from the Compass
✅ level=2, idx=6: 'Opening a

In [5]:
from undetected_chromedriver import Chrome as uc_Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ⬇️ CSV başlat
df = pd.DataFrame(columns=["title", "content"])

# 🔍 İçerik ve breadcrumb çek
def get_breadcrumb_and_content(driver):
    breadcrumb = []
    try:
        li_elements = driver.find_elements(By.XPATH, '//ul[@class="breadcrumb-drop"]//li/a')
        breadcrumb += [li.text.strip() for li in li_elements if li.text.strip()]
    except:
        pass
    try:
        nav_items = driver.find_elements(By.XPATH, '//div[@class="breadcrumb-item nav-item"]')
        breadcrumb += [nav.text.strip() for nav in nav_items if nav.text.strip()]
    except:
        pass
    breadcrumb_str = " > ".join(breadcrumb)
    print("➡️ Breadcrumb:", breadcrumb_str)

    try:
        content_text = driver.find_element(By.XPATH, '//div[@class="content"]').text.strip()
    except:
        content_text = ""
    df.loc[len(df)] = [breadcrumb_str, content_text]
    print(f"💾 Kaydedildi: {breadcrumb_str[:50]}...")

# 🔁 Belirli bir li > a öğesine tıkla ve geri dön
def click_nested_index(parent, level, idx):
    li = parent.find_elements(By.XPATH, f'.//ul[@level="{level}"]/li')[idx]
    link = li.find_element(By.XPATH, './/a[@class="toc-item"]')
    driver.execute_script("arguments[0].scrollIntoView(true);", link)
    time.sleep(1)
    link.click()
    print(f"✅ level={level}, idx={idx} tıklandı: {link.text.strip()}")
    time.sleep(2)
    get_breadcrumb_and_content(driver)
    return li

# ✅ UC Driver başlat
driver = uc_Chrome()
print("✅ UC Driver başlatıldı.")
driver.get("https://help.3ds.com/")
print("✅ Siteye gidildi.")

# Çerezleri kabul et
try:
    accept_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Accept All")]'))
    )
    accept_all_button.click()
    print("✅ 'Accept All' tıklandı.")
except:
    pass
time.sleep(2)

# Giriş işlemleri
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys("irembircan")
print("✅ Kullanıcı adı girildi.")
time.sleep(1)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main-content"]/div[2]/div[1]/div[2]/form/div/input'))).click()
print("✅ Continue tıklandı.")
time.sleep(2)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password"))).send_keys("Fnss2025!")
print("✅ Şifre girildi.")
time.sleep(1)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//input[@value="Log in"]'))).click()
print("✅ Log in tıklandı.")
time.sleep(25)

# ➤ Navigation başlangıcı
menu_item = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f'//li[@data-location="8"]'))
)
driver.execute_script("arguments[0].scrollIntoView(true);", menu_item)
time.sleep(1)
menu_item.click()
print("✅ data-location=8 tıklandı.")
time.sleep(2)

# ➤ Seviye seviye gezinme
lvl2 = driver.find_elements(By.XPATH, '//li[@data-location="8"]//ul[@level="2"]/li')[0]
lvl2.find_element(By.XPATH, './/a[@class="toc-item"]').click()
time.sleep(2)
get_breadcrumb_and_content(driver)

lvl3 = click_nested_index(lvl2, 3, 2)
lvl4 = click_nested_index(lvl3, 4, 2)
lvl5 = click_nested_index(lvl4, 5, 2)
lvl6 = click_nested_index(lvl5, 6, 1)

# 🔽 Level 6'dan sonra level 7 alt başlıklarını tarama
level7_items = lvl6.find_elements(By.XPATH, './/ul[@level="7"]/li')
print(f"🔍 level=7 içinde {len(level7_items)} item bulundu.")

for idx, li in enumerate(level7_items):
    try:
        link = li.find_element(By.XPATH, './/a[@class="toc-item"]')
        driver.execute_script("arguments[0].scrollIntoView(true);", link)
        time.sleep(0.5)
        link.click()
        print(f"✅ level=7 idx={idx} tıklandı: {link.text.strip()}")
        time.sleep(1)
        get_breadcrumb_and_content(driver)
    except Exception as e:
        print(f"⚠️ level=7 idx={idx} tıklanamadı: {e}")
        continue

# 🚪 Kapat
driver.quit()
print("✅ Driver kapatıldı.")
df.to_csv("veri.csv", index=False)
print("✅ veri.csv kaydedildi.")


✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue tıklandı.
✅ Şifre girildi.
✅ Log in tıklandı.
✅ data-location=8 tıklandı.
➡️ Breadcrumb: Installation and Setup > 3DEXPERIENCE Platform
💾 Kaydedildi: Installation and Setup > 3DEXPERIENCE Platform...
✅ level=3, idx=2 tıklandı: 3DEXPERIENCE Configuration and Customization
➡️ Breadcrumb: Installation and Setup > 3DEXPERIENCE Platform > 3DEXPERIENCE Configuration and Customization
💾 Kaydedildi: Installation and Setup > 3DEXPERIENCE Platform > 3...
✅ level=4, idx=2 tıklandı: 3DSpace Data Model
➡️ Breadcrumb: Installation and Setup > 3DEXPERIENCE Platform > 3DEXPERIENCE Configuration and Customization > 3DSpace Data Model
💾 Kaydedildi: Installation and Setup > 3DEXPERIENCE Platform > 3...
✅ level=5, idx=2 tıklandı: Unified Typing Reference Dictionary
➡️ Breadcrumb: Installation and Setup > 3DEXPERIENCE Platform > 3DEXPERIENCE Configuration and Customization > 3DSpace Data Model > Unified T

In [2]:
import pandas as pd

# CSV'yi oku
df = pd.read_csv("veri.csv")

# Satır sonlarını boşlukla değiştir
df['content'] = df['content'].astype(str).str.replace(r'\s*\n\s*', ' ', regex=True)

# İsteğe bağlı: title'da da \n varsa onları da düzelt
df['title'] = df['title'].astype(str).str.replace(r'\s*\n\s*', ' ', regex=True)

# Yeni CSV'ye kaydet
df.to_csv("veri.csv", index=False)

print("✅ Tüm içerikler tek satıra dönüştürüldü. 'veri_clean.csv' dosyasına kaydedildi.")


✅ Tüm içerikler tek satıra dönüştürüldü. 'veri_clean.csv' dosyasına kaydedildi.


In [10]:
from undetected_chromedriver import Chrome as uc_Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# UC Driver başlatma
driver = uc_Chrome()
print("✅ UC Driver başlatıldı.")

# İstenen sayfaya git
driver.get("https://help.3ds.com/")
print("✅ Siteye gidildi.")

# Accept All butonuna tıkla
try:
    accept_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Accept All")]'))
    )
    accept_all_button.click()
    print("✅ 'Accept All' butonuna tıklandı.")
except Exception as e:
    print(f"⚠ 'Accept All' bulunamadı veya tıklanamadı: {e}")

time.sleep(2)

# Username alanına "kullaniciadi" yaz
try:
    username_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "username"))
    )
    username_input.send_keys("irembircan")
    print("✅ Kullanıcı adı girildi.")
except Exception as e:
    print(f"⚠ Kullanıcı adı alanı bulunamadı: {e}")

time.sleep(1)

# Continue butonuna tıkla
try:
    continue_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="main-content"]/div[2]/div[1]/div[2]/form/div/input'))
    )
    continue_button.click()
    print("✅ Continue butonuna tıklandı.")
except Exception as e:
    print(f"⚠ Continue butonuna tıklanamadı: {e}")

time.sleep(2)

# Password alanına "sifre" yaz
try:
    password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "password"))
    )
    password_input.send_keys("Fnss2025!")
    print("✅ Şifre girildi.")
except Exception as e:
    print(f"⚠ Şifre alanı bulunamadı: {e}")

time.sleep(1)

# Log in butonuna tıkla
try:
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//input[@value="Log in"]'))
    )
    login_button.click()
    print("✅ Log in butonuna tıklandı.")
except Exception as e:
    print(f"⚠ Log in butonuna tıklanamadı: {e}")

# Biraz bekle
time.sleep(30)


df = pd.DataFrame(columns=["konum", "icerik"])  

def get_breadcrumb_and_content(driver):
    breadcrumb = []
    try:
        li_elements = driver.find_elements(By.XPATH, '//ul[@class="breadcrumb-drop"]//li/a')
        for li in li_elements:
            text = li.text.strip()
            if text:
                breadcrumb.append(text)
    except Exception as e:
        print(f"⚠ breadcrumb ul okunamadı: {e}")

    try:
        nav_items = driver.find_elements(By.XPATH, '//div[@class="breadcrumb-item nav-item"]')
        for nav in nav_items:
            text = nav.text.strip()
            if text:
                breadcrumb.append(text)
    except Exception as e:
        print(f"⚠ breadcrumb nav okunamadı: {e}")

    breadcrumb_str = " > ".join(breadcrumb)
    print("➡️ Breadcrumb yolu:", breadcrumb_str)

    # 🔥 content'i al
    content_text = ""
    try:
        content_div = driver.find_element(By.XPATH, '//div[@class="content"]')
        content_text = content_div.text.strip()
    except Exception as e:
        print(f"⚠ content div okunamadı: {e}")

    # ➕ DF'e ekle
    df.loc[len(df)] = [breadcrumb_str, content_text]


def crawl_ul(driver, ul_xpath, level=2):
    try:
        sub_items = driver.find_elements(By.XPATH, f'{ul_xpath}//li')
        print(f"🔍 level={level} içinde {len(sub_items)} li bulundu.")

        idx = 0
        while idx < len(sub_items):
            try:
                li = sub_items[idx]
                link = li.find_element(By.XPATH, './/a[@class="toc-item"][1]')
                
                driver.execute_script("arguments[0].scrollIntoView(true);", link)
                time.sleep(0.5)
                link.click()
                print(f"✅ level={level}, idx={idx+1}: '{link.text.strip()}' tıklandı.")

                time.sleep(1)
                get_breadcrumb_and_content(driver)
                time.sleep(0.5)

                next_level = level + 1
                sub_uls = li.find_elements(By.XPATH, f'.//ul[@level="{next_level}"]')
                for sub_ul in sub_uls:
                    sub_ul_xpath = get_xpath(sub_ul)
                    crawl_ul(driver, sub_ul_xpath, next_level)

                driver.back()
                time.sleep(0.5)

                idx += 1

            except Exception as e:
                print(f"⚠ level={level}, idx={idx+1} tıklanamadı: {e}")
                idx += 1

    except Exception as e:
        print(f"🚨 level={level} ul işlenirken hata: {e}")

def get_xpath(el):
    path = ''
    while el.tag_name != 'html':
        siblings = el.find_elements(By.XPATH, f'./preceding-sibling::{el.tag_name}')
        idx = len(siblings) + 1
        path = f'/{el.tag_name}[{idx}]' + path
        el = el.find_element(By.XPATH, '..')
    return '/html' + path


data_loc = 4
while data_loc <= 4:
    try:
        menu_item = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f'//li[@data-location="{str(data_loc)}"]'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", menu_item)
        time.sleep(0.5)
        menu_item.click()
        print(f"✅ data-location={data_loc} tıklandı.")
        time.sleep(1)

        # ➡ burada özel gezinme
        level2_items = driver.find_elements(By.XPATH, f'//li[@data-location="{data_loc}"]//ul[@level="2"]/li')
        if len(level2_items) > 2:
            li3 = level2_items[2]
            link3 = li3.find_element(By.XPATH, './/a[@class="toc-item"][1]')
            driver.execute_script("arguments[0].scrollIntoView(true);", link3)
            time.sleep(0.5)
            link3.click()
            print(f"✅ level=2 idx=3 tıklandı: '{link3.text.strip()}'")
            time.sleep(1)

            level3_items = li3.find_elements(By.XPATH, './/ul[@level="3"]/li')
            if len(level3_items) > 0:
                li1 = level3_items[0]
                link1 = li1.find_element(By.XPATH, './/a[@class="toc-item"][1]')
                driver.execute_script("arguments[0].scrollIntoView(true);", link1)
                time.sleep(0.5)
                link1.click()
                print(f"✅ level=3 idx=1 tıklandı: '{link1.text.strip()}'")
                time.sleep(1)

                level4_items = li1.find_elements(By.XPATH, './/ul[@level="4"]/li')
                print(f"🔍 level=4 içinde {len(level4_items)} li bulundu.")

                for idx in range(5, len(level4_items)):
                    try:
                        li = level4_items[idx]
                        link = li.find_element(By.XPATH, './/a[@class="toc-item"][1]')
                        driver.execute_script("arguments[0].scrollIntoView(true);", link)
                        time.sleep(0.5)
                        link.click()
                        print(f"✅ level=4 idx={idx+1} tıklandı: '{link.text.strip()}'")
                        time.sleep(1)
                        get_breadcrumb_and_content(driver)
                        time.sleep(0.5)
                    except Exception as e:
                        print(f"⚠ level=4 idx={idx+1} tıklanamadı: {e}")
        else:
            print("⚠ level=2'de yeterli sayıda item yok.")

    except Exception as e:
        print(f"⚠ data-location={data_loc} bulunamadı veya tıklanamadı: {e}")

    data_loc += 1

driver.quit()
print("✅ İşlem tamam, driver kapatıldı.")

# ➕ CSV'ye kaydet
df.to_csv("data.csv", index=False)
print("✅ data.csv kaydedildi.")


✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' butonuna tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue butonuna tıklandı.
✅ Şifre girildi.
✅ Log in butonuna tıklandı.
✅ data-location=4 tıklandı.
✅ level=2 idx=3 tıklandı: '3DEXPERIENCE with SOLIDWORKS'
✅ level=3 idx=1 tıklandı: '3DEXPERIENCE Simulation for SOLIDWORKS Users'
🔍 level=4 içinde 8 li bulundu.
✅ level=4 idx=6 tıklandı: 'Feature Transfer from SOLIDWORKS to 3DEXPERIENCE Simulations'
➡️ Breadcrumb yolu: 3D Modeling > 3DEXPERIENCE with SOLIDWORKS > 3DEXPERIENCE Simulation for SOLIDWORKS Users > Feature Transfer from SOLIDWORKS to 3DEXPERIENCE Simulations
✅ level=4 idx=7 tıklandı: 'Materials'
➡️ Breadcrumb yolu: 3D Modeling > 3DEXPERIENCE with SOLIDWORKS > 3DEXPERIENCE Simulation for SOLIDWORKS Users > Materials
✅ level=4 idx=8 tıklandı: 'Sitemap'
➡️ Breadcrumb yolu: 3D Modeling > 3DEXPERIENCE with SOLIDWORKS > 3DEXPERIENCE Simulation for SOLIDWORKS Users > Sitemap
✅ İşlem tamam, driver kapatıldı.
✅ data.csv kaydedildi.


In [8]:
from undetected_chromedriver import Chrome as uc_Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ➡️ CSV'yi oku
try:
    df = pd.read_csv("data.csv")
    print("✅ Mevcut data.csv yüklendi.")
except FileNotFoundError:
    df = pd.DataFrame(columns=["konum", "icerik"])
    print("✅ Yeni dataframe başlatıldı.")

def get_breadcrumb_and_content(driver):
    breadcrumb = []
    try:
        li_elements = driver.find_elements(By.XPATH, '//ul[@class="breadcrumb-drop"]//li/a')
        for li in li_elements:
            text = li.text.strip()
            if text:
                breadcrumb.append(text)
    except:
        pass
    try:
        nav_items = driver.find_elements(By.XPATH, '//div[@class="breadcrumb-item nav-item"]')
        for nav in nav_items:
            text = nav.text.strip()
            if text:
                breadcrumb.append(text)
    except:
        pass
    breadcrumb_str = " > ".join(breadcrumb)
    print("➡️ Breadcrumb yolu:", breadcrumb_str)

    content_text = ""
    try:
        content_div = driver.find_element(By.XPATH, '//div[@class="content"]')
        content_text = content_div.text.strip()
    except:
        pass

    df.loc[len(df)] = [breadcrumb_str, content_text]
    print(f"💾 Kaydedildi: {breadcrumb_str[:30]}...")
    df.to_csv("data.csv", index=False)

def get_xpath(el):
    path = ''
    while el.tag_name != 'html':
        siblings = el.find_elements(By.XPATH, f'./preceding-sibling::{el.tag_name}')
        idx = len(siblings) + 1
        path = f'/{el.tag_name}[{idx}]' + path
        el = el.find_element(By.XPATH, '..')
    return '/html' + path

def crawl_ul(driver, ul_xpath, level=3):
    try:
        sub_items = driver.find_elements(By.XPATH, f'{ul_xpath}//li')
        print(f"🔍 level={level} içinde {len(sub_items)} li bulundu.")
        idx = 0
        while idx < len(sub_items):
            try:
                li = sub_items[idx]
                link = li.find_element(By.XPATH, './/a[@class="toc-item"][1]')
                driver.execute_script("arguments[0].scrollIntoView(true);", link)
                time.sleep(0.5)
                link.click()
                print(f"✅ level={level}, idx={idx+1}: '{link.text.strip()}' tıklandı.")
                time.sleep(0.5)
                get_breadcrumb_and_content(driver)
                time.sleep(0.5)

                next_level = level + 1
                sub_uls = li.find_elements(By.XPATH, f'.//ul[@level="{next_level}"]')
                for sub_ul in sub_uls:
                    sub_ul_xpath = get_xpath(sub_ul)
                    crawl_ul(driver, sub_ul_xpath, next_level)

                driver.back()
                time.sleep(0.5)
                idx += 1
            except Exception as e:
                print(f"⚠ level={level}, idx={idx+1} tıklanamadı: {e}")
                idx += 1
    except Exception as e:
        print(f"🚨 level={level} ul işlenirken hata: {e}")

# ✅ UC Driver başlat
driver = uc_Chrome()
print("✅ UC Driver başlatıldı.")
driver.get("https://help.3ds.com/")
print("✅ Siteye gidildi.")

# ➡️ login işlemleri
try:
    accept_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Accept All")]'))
    )
    accept_all_button.click()
    print("✅ 'Accept All' tıklandı.")
except:
    pass
time.sleep(2)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys("irembircan")
print("✅ Kullanıcı adı girildi.")
time.sleep(1)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main-content"]/div[2]/div[1]/div[2]/form/div/input'))).click()
print("✅ Continue tıklandı.")
time.sleep(2)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password"))).send_keys("Fnss2025!")
print("✅ Şifre girildi.")
time.sleep(1)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//input[@value="Log in"]'))).click()
print("✅ Log in tıklandı.")
time.sleep(30)

# ➡️ 3D Modeling Core > Part Design aç
menu_item = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//a[normalize-space()="3D Modeling Core"]/parent::li'))
)
driver.execute_script("arguments[0].scrollIntoView(true);", menu_item)
time.sleep(0.5)
menu_item.click()
print("✅ '3D Modeling Core' tıklandı.")
time.sleep(2)

part_design_li = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[normalize-space()="Part Design"]/parent::li'))
)
driver.execute_script("arguments[0].scrollIntoView(true);", part_design_li)
time.sleep(0.5)
part_design_li.click()
print("✅ 'Part Design' tıklandı.")
time.sleep(2)

# ➡️ Part Design altındaki başlıkları bul
part_design_items = part_design_li.find_elements(By.XPATH, './/ul[@level="3"]/li')
print(f"🔍 Part Design içinde {len(part_design_items)} alt başlık bulundu.")

# ➡️ 6. indexten başlayarak sonuna kadar tara
for idx in range(6, len(part_design_items)):
    try:
        li = part_design_items[idx]
        link = li.find_element(By.XPATH, './/a[@class="toc-item"][1]')
        driver.execute_script("arguments[0].scrollIntoView(true);", link)
        time.sleep(0.5)
        link.click()
        print(f"✅ Part Design idx={idx}: '{link.text.strip()}' tıklandı.")
        time.sleep(1)

        get_breadcrumb_and_content(driver)
        time.sleep(0.5)

        sub_uls = li.find_elements(By.XPATH, './/ul')
        for sub_ul in sub_uls:
            sub_ul_xpath = get_xpath(sub_ul)
            crawl_ul(driver, sub_ul_xpath, 4)

        driver.back()
        time.sleep(1)
    except Exception as e:
        print(f"⚠ Part Design idx={idx} işlenirken hata: {e}")

# ➡️ Bitti
driver.quit()
print("✅ İşlem tamam, driver kapatıldı.")
df.to_csv("data.csv", index=False)
print("✅ data.csv son haliyle kaydedildi.")


✅ Mevcut data.csv yüklendi.
✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue tıklandı.
✅ Şifre girildi.
✅ Log in tıklandı.


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0xe344a3+62419]
	GetHandleVerifier [0x0xe344e4+62484]
	(No symbol) [0x0xc72133]
	(No symbol) [0x0xcba8fe]
	(No symbol) [0x0xcbac9b]
	(No symbol) [0x0xd03052]
	(No symbol) [0x0xcdf4b4]
	(No symbol) [0x0xd0087a]
	(No symbol) [0x0xcdf266]
	(No symbol) [0x0xcae852]
	(No symbol) [0x0xcaf6f4]
	GetHandleVerifier [0x0x10a4793+2619075]
	GetHandleVerifier [0x0x109fbaa+2599642]
	GetHandleVerifier [0x0xe5b04a+221050]
	GetHandleVerifier [0x0xe4b2c8+156152]
	GetHandleVerifier [0x0xe51c7d+183213]
	GetHandleVerifier [0x0xe3c388+94904]
	GetHandleVerifier [0x0xe3c512+95298]
	GetHandleVerifier [0x0xe2766a+9626]
	BaseThreadInitThunk [0x0x759f7ba9+25]
	RtlInitializeExceptionChain [0x0x7700c3ab+107]
	RtlClearBits [0x0x7700c32f+191]


In [5]:
from undetected_chromedriver import Chrome as uc_Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ➡️ CSV'yi oku
try:
    df = pd.read_csv("data.csv")
    print("✅ Mevcut data.csv yüklendi.")
except FileNotFoundError:
    df = pd.DataFrame(columns=["konum", "icerik"])
    print("✅ Yeni dataframe başlatıldı.")

def get_breadcrumb_and_content(driver):
    breadcrumb = []
    try:
        li_elements = driver.find_elements(By.XPATH, '//ul[@class="breadcrumb-drop"]//li/a')
        for li in li_elements:
            text = li.text.strip()
            if text:
                breadcrumb.append(text)
    except:
        pass
    try:
        nav_items = driver.find_elements(By.XPATH, '//div[@class="breadcrumb-item nav-item"]')
        for nav in nav_items:
            text = nav.text.strip()
            if text:
                breadcrumb.append(text)
    except:
        pass
    breadcrumb_str = " > ".join(breadcrumb)
    print("➡️ Breadcrumb yolu:", breadcrumb_str)

    content_text = ""
    try:
        content_div = driver.find_element(By.XPATH, '//div[@class="content"]')
        content_text = content_div.text.strip()
    except:
        pass

    df.loc[len(df)] = [breadcrumb_str, content_text]
    print(f"💾 Kaydedildi: {breadcrumb_str[:30]}...")
    df.to_csv("data.csv", index=False)

def get_xpath(el):
    path = ''
    while el.tag_name != 'html':
        siblings = el.find_elements(By.XPATH, f'./preceding-sibling::{el.tag_name}')
        idx = len(siblings) + 1
        path = f'/{el.tag_name}[{idx}]' + path
        el = el.find_element(By.XPATH, '..')
    return '/html' + path

def crawl_ul(driver, ul_xpath, level=3):
    try:
        sub_items = driver.find_elements(By.XPATH, f'{ul_xpath}//li')
        print(f"🔍 level={level} içinde {len(sub_items)} li bulundu.")
        idx = 0
        while idx < len(sub_items):
            try:
                li = sub_items[idx]
                link = li.find_element(By.XPATH, './/a[@class="toc-item"][1]')
                driver.execute_script("arguments[0].scrollIntoView(true);", link)
                time.sleep(0.5)
                link.click()
                print(f"✅ level={level}, idx={idx+1}: '{link.text.strip()}' tıklandı.")
                time.sleep(0.5)
                get_breadcrumb_and_content(driver)
                time.sleep(0.5)

                next_level = level + 1
                sub_uls = li.find_elements(By.XPATH, f'.//ul[@level="{next_level}"]')
                for sub_ul in sub_uls:
                    sub_ul_xpath = get_xpath(sub_ul)
                    crawl_ul(driver, sub_ul_xpath, next_level)

                driver.back()
                time.sleep(0.5)
                idx += 1
            except Exception as e:
                print(f"⚠ level={level}, idx={idx+1} tıklanamadı: {e}")
                idx += 1
    except Exception as e:
        print(f"🚨 level={level} ul işlenirken hata: {e}")

# ✅ UC Driver başlat
driver = uc_Chrome()
print("✅ UC Driver başlatıldı.")
driver.get("https://help.3ds.com/")
print("✅ Siteye gidildi.")

# ➡️ login işlemleri
try:
    accept_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Accept All")]'))
    )
    accept_all_button.click()
    print("✅ 'Accept All' tıklandı.")
except:
    pass
time.sleep(2)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys("irembircan")
print("✅ Kullanıcı adı girildi.")
time.sleep(1)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main-content"]/div[2]/div[1]/div[2]/form/div/input'))).click()
print("✅ Continue tıklandı.")
time.sleep(2)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password"))).send_keys("Fnss2025!")
print("✅ Şifre girildi.")
time.sleep(1)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//input[@value="Log in"]'))).click()
print("✅ Log in tıklandı.")
time.sleep(30)

# ➡️ data-location=4 (Enterprise Knowledge Language)
menu_item = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//li[@data-location="4"]'))
)
driver.execute_script("arguments[0].scrollIntoView(true);", menu_item)
time.sleep(0.5)
menu_item.click()
print("✅ data-location=4 tıklandı.")
time.sleep(2)

# ➡️ doğrudan level=2 idx=2
level2_items = driver.find_elements(By.XPATH, '//li[@data-location="4"]//ul[@level="2"]//li')
li_lvl2 = level2_items[3]
link_lvl2 = li_lvl2.find_element(By.XPATH, './/a[@class="toc-item"][1]')
driver.execute_script("arguments[0].scrollIntoView(true);", link_lvl2)
time.sleep(1)
link_lvl2.click()
print(f"✅ level=2 idx=2 tıklandı: '{link_lvl2.text.strip()}'")
time.sleep(2)

# ➡️ altındaki UL leri tara
sub_uls_lvl3 = li_lvl2.find_elements(By.XPATH, './/ul[@level="3"]')
for sub_ul in sub_uls_lvl3:
    sub_ul_xpath = get_xpath(sub_ul)
    crawl_ul(driver, sub_ul_xpath, 3)

# ➡️ Bitti
driver.quit()
print("✅ İşlem tamam, driver kapatıldı.")
df.to_csv("data.csv", index=False)
print("✅ data.csv son haliyle kaydedildi.")

✅ Mevcut data.csv yüklendi.
✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue tıklandı.
✅ Şifre girildi.
✅ Log in tıklandı.
✅ data-location=4 tıklandı.
✅ level=2 idx=2 tıklandı: '3D Modeling Core'
🔍 level=3 içinde 50 li bulundu.
✅ level=3, idx=1: 'Natural Shape' tıklandı.
➡️ Breadcrumb yolu: 3D Modeling > 3D Modeling Core > Natural Shape
💾 Kaydedildi: 3D Modeling > 3D Modeling Core...
🔍 level=4 içinde 39 li bulundu.
✅ level=4, idx=1: 'What's New' tıklandı.
➡️ Breadcrumb yolu: 3D Modeling > 3D Modeling Core > Natural Shape > What's New
💾 Kaydedildi: 3D Modeling > 3D Modeling Core...
🔍 level=5 içinde 9 li bulundu.
✅ level=5, idx=1: 'R2024x FD02' tıklandı.
➡️ Breadcrumb yolu: 3D Modeling > 3D Modeling Core > Natural Shape > What's New > R2024x FD02
💾 Kaydedildi: 3D Modeling > 3D Modeling Core...
🔍 level=6 içinde 2 li bulundu.
✅ level=6, idx=1: 'Simplified Color Coding' tıklandı.
➡️ Breadcrumb yolu: 3D Modeling > 3D Modeling Core > Natur

✅ Mevcut data.csv yüklendi.
✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue tıklandı.
✅ Şifre girildi.
✅ Log in tıklandı.


✅ Mevcut data.csv yüklendi.
✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue tıklandı.
✅ Şifre girildi.
✅ Log in tıklandı.
✅ data-location=4 tıklandı.
✅ level=2 idx=0 tıklandı: '3DPlay'
🔍 level=3 içinde 2 li bulundu.
✅ UC Driver kapatıldı.
✅ data.csv son haliyle kaydedildi.


✅ Mevcut data.csv yüklendi.
✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue tıklandı.
✅ Şifre girildi.
✅ Log in tıklandı.
✅ Enterprise Knowledge Language tıklandı.
✅ Knowledge Packages içinde idx=11 tıklandı.
🔍 level=4 içinde 3 li bulundu.
✅ level=4, idx=1: 'About the EKL Client and Server' tıklandı.
➡️ Breadcrumb yolu: 3DEXPERIENCE > Enterprise Knowledge Language > Server Types and Functions > About the EKL Client and Server
💾 Kaydedildi: 3DEXPERIENCE > Enterprise Know...
✅ level=4, idx=2: 'About EKL Server Functions' tıklandı.
➡️ Breadcrumb yolu: 3DEXPERIENCE > Enterprise Knowledge Language > Server Types and Functions > About EKL Server Functions
💾 Kaydedildi: 3DEXPERIENCE > Enterprise Know...
🔍 level=5 içinde 2 li bulundu.
✅ level=5, idx=1: 'Base PLM Type: PLMEntity' tıklandı.
➡️ Breadcrumb yolu: 3DEXPERIENCE > Enterprise Knowledge Language > Server Types and Functions > About EKL Server Functions > Base PLM Type: PLMEntity
💾 K

In [ ]:
from undetected_chromedriver import Chrome as uc_Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ➡️ CSV'yi önce oku
try:
    df = pd.read_csv("data.csv")
    print("✅ Mevcut data.csv yüklendi.")
except FileNotFoundError:
    df = pd.DataFrame(columns=["konum", "icerik"])
    print("✅ Yeni dataframe başlatıldı.")

def get_breadcrumb_and_content(driver):
    breadcrumb = []
    try:
        li_elements = driver.find_elements(By.XPATH, '//ul[@class="breadcrumb-drop"]//li/a')
        for li in li_elements:
            text = li.text.strip()
            if text:
                breadcrumb.append(text)
    except:
        pass
    try:
        nav_items = driver.find_elements(By.XPATH, '//div[@class="breadcrumb-item nav-item"]')
        for nav in nav_items:
            text = nav.text.strip()
            if text:
                breadcrumb.append(text)
    except:
        pass
    breadcrumb_str = " > ".join(breadcrumb)
    print("➡️ Breadcrumb yolu:", breadcrumb_str)

    content_text = ""
    try:
        content_div = driver.find_element(By.XPATH, '//div[@class="content"]')
        content_text = content_div.text.strip()
    except:
        pass

    df.loc[len(df)] = [breadcrumb_str, content_text]
    print(f"💾 Kaydedildi: {breadcrumb_str[:30]}...")

def get_xpath(el):
    path = ''
    while el.tag_name != 'html':
        siblings = el.find_elements(By.XPATH, f'./preceding-sibling::{el.tag_name}')
        idx = len(siblings) + 1
        path = f'/{el.tag_name}[{idx}]' + path
        el = el.find_element(By.XPATH, '..')
    return '/html' + path

def crawl_ul(driver, ul_xpath, level):
    try:
        sub_items = driver.find_elements(By.XPATH, f'{ul_xpath}//li')
        print(f"🔍 level={level} içinde {len(sub_items)} li bulundu.")
        idx = 0
        while idx < len(sub_items):
            try:
                li = sub_items[idx]
                link = li.find_element(By.XPATH, './/a[@class="toc-item"][1]')
                driver.execute_script("arguments[0].scrollIntoView(true);", link)
                time.sleep(0.5)
                link.click()
                print(f"✅ level={level}, idx={idx+1}: '{link.text.strip()}' tıklandı.")
                time.sleep(0.5)
                get_breadcrumb_and_content(driver)
                time.sleep(0.5)

                # 🚀 Özel bitirme koşulu
                if level == 2 and idx == 2:
                    df.to_csv("data.csv", index=False)
                    print("✅ level=2 idx=2'ye gelindi, data.csv kaydedildi ve program sonlandırıldı.")
                    driver.quit()
                    exit()

                next_level = level + 1
                sub_uls = li.find_elements(By.XPATH, f'.//ul[@level="{next_level}"]')
                for sub_ul in sub_uls:
                    sub_ul_xpath = get_xpath(sub_ul)
                    crawl_ul(driver, sub_ul_xpath, next_level)
                driver.back()
                time.sleep(0.5)
                idx += 1
            except Exception as e:
                print(f"⚠ level={level}, idx={idx+1} tıklanamadı: {e}")
                idx += 1
    except Exception as e:
        print(f"🚨 level={level} ul işlenirken hata: {e}")

# 🔥 UC DRIVER başlat
driver = uc_Chrome()
print("✅ UC Driver başlatıldı.")
driver.get("https://help.3ds.com/")
print("✅ Siteye gidildi.")

# Login işlemleri
try:
    accept_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Accept All")]'))
    )
    accept_all_button.click()
    print("✅ 'Accept All' tıklandı.")
except:
    pass

time.sleep(2)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys("irembircan")
print("✅ Kullanıcı adı girildi.")
time.sleep(1)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main-content"]/div[2]/div[1]/div[2]/form/div/input'))).click()
print("✅ Continue tıklandı.")
time.sleep(2)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password"))).send_keys("Fnss2025!")
print("✅ Şifre girildi.")
time.sleep(1)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//input[@value="Log in"]'))).click()
print("✅ Log in tıklandı.")
time.sleep(30)

# ➡️ data-location=3
menu_item = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//li[@data-location="3"]'))
)
driver.execute_script("arguments[0].scrollIntoView(true);", menu_item)
time.sleep(1)
menu_item.click()
print("✅ data-location=3 tıklandı.")
time.sleep(2)

# ➡️ level=2, idx=1
level2_items = driver.find_elements(By.XPATH, '//li[@data-location="3"]//ul[@level="2"]//li')
level2_items[1].find_element(By.XPATH, './/a[@class="toc-item"][1]').click()
print("✅ level=2, idx=1 tıklandı.")
time.sleep(2)

# ➡️ level=3, idx=2
level3_items = driver.find_elements(By.XPATH, '//ul[@level="3"]//li')
level3_items[4].find_element(By.XPATH, './/a[@class="toc-item"][1]').click()
print("✅ level=3, idx=2 tıklandı.")
time.sleep(2)


# ➡️ Artık buradan itibaren crawl_ul
sub_uls = level3_items[4].find_elements(By.XPATH, './/ul[@level="5"]')
for sub_ul in sub_uls:
    sub_ul_xpath = get_xpath(sub_ul)
    crawl_ul(driver, sub_ul_xpath, 5)

driver.quit()
print("✅ UC Driver kapatıldı.")
df.to_csv("data.csv", index=False)
print("✅ data.csv son haliyle kaydedildi.")


✅ Mevcut data.csv yüklendi.
✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue tıklandı.
✅ Şifre girildi.
✅ Log in tıklandı.
✅ data-location=3 tıklandı.
✅ level=2, idx=1 tıklandı.
✅ level=3, idx=2 tıklandı.
✅ UC Driver kapatıldı.
✅ data.csv son haliyle kaydedildi.


In [ ]:

Dress-Up Features

✅ Mevcut data.csv yüklendi.
✅ UC Driver başlatıldı.
✅ Siteye gidildi.
✅ 'Accept All' tıklandı.
✅ Kullanıcı adı girildi.
✅ Continue tıklandı.
✅ Şifre girildi.
✅ Log in tıklandı.
✅ data-location=3 tıklandı.
✅ level=2, idx=4 tıklandı.
✅ level=3, idx=10 tıklandı.
🔍 level=4 içinde 3 li bulundu.
✅ level=4, idx=1: 'About the EKL Client and Server' tıklandı.
➡️ Breadcrumb yolu: 3DEXPERIENCE > Enterprise Knowledge Language > Server Types and Functions > About the EKL Client and Server
💾 Kaydedildi: 3DEXPERIENCE > Enterprise Know...
✅ level=4, idx=2: 'About EKL Server Functions' tıklandı.
➡️ Breadcrumb yolu: 3DEXPERIENCE > Enterprise Knowledge Language > Server Types and Functions > About EKL Server Functions
💾 Kaydedildi: 3DEXPERIENCE > Enterprise Know...
✅ level=4, idx=3: 'EKL Functions by Modeler' tıklandı.
➡️ Breadcrumb yolu: 3DEXPERIENCE > Enterprise Knowledge Language > Server Types and Functions > EKL Functions by Modeler
💾 Kaydedildi: 3DEXPERIENCE > Enterprise Know...
🔍 level=5 içinde 7 

In [39]:
print(df.loc[2124, "konum"])

3DEXPERIENCE > PLM Components > Product Modeler > Authoring Operations Availability with Product Modeler > Authoring on Instances


In [6]:
import pandas as pd
df=pd.read_csv("data.csv")
print(df.iloc[-1]["konum"])

3D Modeling > 3D Modeling Core > Part Design > Sketch-Based Features > Solid Combines > About Solid Combines


In [7]:
df.info

<bound method DataFrame.info of                                                  konum  \
0                                 3D Modeling > 3DPlay   
1                           3D Modeling > Globe Viewer   
2    3D Modeling > Globe Viewer > Navigating the Globe   
3                                                  NaN   
4       3D Modeling > 3D Modeling Core > Natural Shape   
..                                                 ...   
451  3D Modeling > 3D Modeling Core > Part Design >...   
452  3D Modeling > 3D Modeling Core > Part Design >...   
453  3D Modeling > 3D Modeling Core > Part Design >...   
454  3D Modeling > 3D Modeling Core > Part Design >...   
455  3D Modeling > 3D Modeling Core > Part Design >...   

                                                icerik  
0    3DPlay for Desktop apps offer an intuitive nav...  
1    Globe Viewer is a web app allowing you to disp...  
2    You can navigate the globe by zooming in or ou...  
3                                          

In [4]:
print(df[df['icerik'].isnull()])

                                                 konum icerik
3              Getting Started > Accessing a Dashboard    NaN
4    Getting Started > Opening an App from the Compass    NaN
5    Getting Started > Opening a Native App from a ...    NaN
74   3DEXPERIENCE > 3DEXPERIENCE Platform > 3DEXPER...    NaN
97   3DEXPERIENCE > 3DEXPERIENCE Platform > Using 3...    NaN
98   3DEXPERIENCE > 3DEXPERIENCE Platform > Using 3...    NaN
99   3DEXPERIENCE > 3DEXPERIENCE Platform > Using 3...    NaN
122  3DEXPERIENCE > 3DEXPERIENCE Platform > Collabo...    NaN
123  3DEXPERIENCE > 3DEXPERIENCE Platform > Collabo...    NaN
138  3DEXPERIENCE > 3DEXPERIENCE Platform > Using D...    NaN
139  3DEXPERIENCE > 3DEXPERIENCE Platform > Using D...    NaN
140  3DEXPERIENCE > 3DEXPERIENCE Platform > Using D...    NaN
141  3DEXPERIENCE > 3DEXPERIENCE Platform > Using D...    NaN
142  3DEXPERIENCE > 3DEXPERIENCE Platform > Using D...    NaN
143  3DEXPERIENCE > 3DEXPERIENCE Platform > Using D...    NaN
144  3DE